In [38]:
import json
from pprint import pprint
import pandas as pd
import geojson
import tempfile

In [39]:
src_data_caba = '../data_caba.json'
dst_data_caba = '../join_data_caba.json'

src_data_gba  = '../data_gba.json'
dst_data_gba  = '../join_data_gba.json'

In [112]:
def estandarizar_keys_censo(key):
    new_key = key.lower()
    if (new_key == 'mujeres') : new_key = 'mujer'
    if (new_key == 'varones') : new_key = 'varon'
    if (new_key == 'totalpobl') : new_key = 'tot_pob'
    if (new_key == 'viviendasp') : new_key = 'viv_part'
    return new_key
    
def generar_data_mapa(src_data, dst_data):
    with open(src_data) as data_file:    
        data = json.load(data_file)
    
    # Estandarizacion de keys
    for row in data['features']:
        row['properties'] = dict(
            (estandarizar_keys_censo(k), v) 
            for k,v in row['properties'].items())

    df = pd.DataFrame(data['features'])

    # LOAD CSV DATA
    edades  = pd.read_csv('../data/csvs_radios/persona/PERSONA-P02.csv', encoding='latin-1')
    hogares = pd.read_csv('../data/csvs_radios/hogar/HOGAR-TOTPERS.csv', encoding='latin-1')

    # MERGE CSV DATA WITH GEOJSON DATA
    info = pd.merge(edades,hogares,on='radio')
    res = df.join(info, lsuffix='link', rsuffix='radio')
    
    # CREATE NEW GEOJSON FILE
    for index, row in res.iterrows():
        row['properties'].update({
            '2 Mujer': row['2 Mujer'], 
            'TOTAL_x' : row['TOTAL_x']
        })
        if(index == 10):
            print(row['properties'])
    # Genero json con datos para el mapa
    with open(dst_data, 'w') as outfile:
          geojson.dump(data, outfile)

In [113]:
generar_data_mapa(src_data_caba, dst_data_caba)

{'area': 58283.5, 'perimeter': 995.418, 'pais0210_': 12, 'pais0210_i': 11, 'prov': '02', 'depto': '013', 'frac': '01', 'radio': '06', 'tipo': 'U', 'link': '020130106', 'varon': 430.0, 'mujer': 452.0, 'tot_pob': 882.0, 'hogares': 386.0, 'viv_part': 519.0, 'viv_part_h': 356.0, '2 Mujer': 182, 'TOTAL_x': 327}


In [114]:
generar_data_mapa(src_data_gba, dst_data_gba)

{'toponimo_i': 334477, 'link': '060700302', 'varon': 126, 'mujer': 121, 'tot_pob': 247, 'hogares': 81, 'viv_part': 145, 'viv_part_h': 76, '2 Mujer': 182, 'TOTAL_x': 327}
